# Import libraries and check the directory

In [ ]:
# Using miniforge for first time...these are the steps to fix the env
'''
# open miniforge prompt, navigate to dir, and create a new env
conda create --name bem_env2 python=3.12
# install ipykernel
conda install -n bem_env2 ipykernel --update-deps --force-reinstall
# install desired libs
conda activate bem_env2
conda install pandas
conda install pyodbc
conda install bs4
conda install dataretrieval
'''

: 

In [3]:
import os # #for working with directories
import pandas as pd #for creating dataframe
import numpy as np #for replacing NaN values in a dataframe
from datetime import datetime #for getting today's date
import pyodbc #working with ODBC databases
from bs4 import BeautifulSoup
import urllib
import dataretrieval.nwis as nwis

# No needed anymore
#import sqlalchemy_access 

In [4]:
# check your current working directory   
print('Old directory:', os.getcwd())

# connect to the shared drive which holds the databases and files (must be connected to VPN)
os.chdir('P:/0083/analysis/DataCompilation/DataCompilationPy/create_site_info_files')
print('New directory:', os.getcwd())

Old directory: c:\Users\bmilinic\OneDrive - DOI\Documents\Python\bemidji
New directory: P:\0083\analysis\DataCompilation\DataCompilationPy\create_site_info_files


In [5]:
#Check to make sure the path exists in current working directory for future use
os.path.exists("../local_access_db/BemidjiMasterSiteData_be.accdb") # ".." means "go back one directory"

True

In [4]:
# If you want every column displayed for the output, delete the pound signs from the code below and run it. Afterwards rerun any dataframe

## pd.set_option('display.max_columns', 0)
## df_bmj3.head()

# bmj3 import

## Meta data

In [6]:
# bmj files are obtained from GWSI. Load the meta into a dataframe
f = r'data_inputs/gwsi_old/bmj3.subf.list_meta.csv'
df_meta = pd.read_csv(f) 
df_meta.head(5)

,FIELD,CODE,DESCRIPTION,LOC,LEN,DB_name
0,1,C004,Source agency code,1,5,GWSI_AgencyCode
1,2,C001,Site ID (station number),6,15,GWSI_USGS_siteno
2,3,C802,Site type code,21,7,GWSI_GWSISiteType
3,4,C023,Primary use of site,28,1,GWSI_GWSIUseOfSite
4,5,C713,Aquifer type code,29,1,GWSI_GWSIAquiferType


## .subf file import and save

In [7]:
# Load GWSI data into a dataframe and rename it
f = r'data_inputs/gwsi_old/bmj3.subf'
#df_bmj3 = pd.read_fwf (f, widths = df_meta.LEN , header=None, names=list(df_meta.DB_name.str.replace('GWSI', 'NWIS'))) # this says (file name, widths are based on a column in df_meta, there's no initial header/col names, the headers/col names should be based on a column in the metadata with an edit)
df_bmj3 = pd.read_fwf (f, widths = df_meta.LEN , header=None, names=list(df_meta.DB_name)) # this says (file name, widths are based on a column in df_meta, there's no initial header/col names, the headers/col names should be based on a column in the metadata)

# Update the Land Surface Altitude column (m) to be rounded to 3 decimal places
df_bmj3['GWSI_LandSurfaceAltitude_mASL_NAVD88'] = df_bmj3['GWSI_LandSurfaceAltitude_ftASL_NAVD88'].mul(0.3048).round(3)

In [8]:
df_bmj3.head(2)

,GWSI_AgencyCode,GWSI_USGS_siteno,GWSI_GWSISiteType,GWSI_GWSIUseOfSite,GWSI_GWSIAquiferType,GWSI_GWSIPrimaryAquifer,GWSI_GWSINationalAquifer,GWSI_TotalWellDepth_ftBLS,GWSI_WellCasingInnerDiameter_inches,GWSI_CasingMaterial,...,GWSI_MP_SequenceNo,GWSI_CONS_RecordType,GWSI_MP_LastUpdate,GWSI_MP_WebReady,GWSI_MP_height_m,GWSI_MP_Altitude_m,GWSI_CONS_SequenceNo,GWSI_HOLE_SequenceNo,GWSI_CSNG_SequenceNo,GWSI_OPEN_SequenceNo
0,USGS,473429095051006,GW,O,U,112GSSG,N100GLCIAL,19.07,0.78,V,...,1.0,MPNT,2.018102e+13,Y,0.34,NaN,1.0,1.0,1.0,1.0
1,USGS,473424095052912,SB-UZ,O,NaN,NaN,NaN,18.09,0.13,V,...,1.0,MPNT,2.020021e+13,Y,0.00,NaN,1.0,1.0,1.0,1.0


In [9]:
# save the output as a file
# Note: Make sure to have the excel file closed or else you CANNOT overwrite the save file
df_bmj3.to_csv("data_inputs/gwsi_old/bmj3_fromPy.csv", index=False) #if I want to get rid of number on the side, use index=False

# bmj.mpnt import 
Repeat the same general process but with different files

In [10]:
f = r'data_inputs/gwsi_old/bmj.mpnt.subf.list_meta.csv'
df_meta = pd.read_csv(f) 

f = r'data_inputs/gwsi_old/bmj.mpnt.subf'
df_mpnt = pd.read_fwf (f, widths = df_meta.LEN , header=None, names=list(df_meta.DB_name))
df_mpnt.to_csv("data_inputs/gwsi_old/bmj_mpnt_fromPy.csv", index=False)

# bmj.rmk import
Repeat again

In [11]:
f = r'data_inputs/gwsi_old/bmj.rmk.subf.list_meta.csv'
df_meta = pd.read_csv(f) 

f = r'data_inputs/gwsi_old/bmj.rmk.subf'
df_rmk = pd.read_fwf (f, widths = df_meta.LEN , header=None, names=list(df_meta.DB_name))
df_rmk.to_csv("data_inputs/gwsi_old/bmj_rmk_fromPy.csv", index=False)

# Referencepoints(2) .csv import
Good MP data

In [10]:
f = r'data_inputs/aquarius/Referencepoints (2).csv'
f = r'data_inputs/aquarius/Referencepoints20241023.csv'
f = r'data_inputs/aquarius/Referencepoints20250130.csv'
df_rp = pd.read_csv(f)
display(df_rp.head(2), df_rp.shape)

,UniqueId,Name,Description,site,StandardIdentifier,IsMeasuredAgainstLocalAssumedDatum,ValidFrom,Unit,Elevation,Uncertainty,Method,MeasurementDirection,Comment,AppliedTime,AppliedByUser,DecommissionedDate,DecommissionedReason
0,0868303bbc6c441982fff320c2d0a44f,MP-1 height,"No MP exists, but 0.00 ft. was populated for f...",473356095043701,NaN,True,0001-01-01T00:00:00.0000000+00:00,ft,0.0,0.05,GNSS3,FromTopToBottom,"No MP exists, but 0.00 ft. was populated for f...",2021-03-02T13:12:52.2229750-06:00,aberg,NaN,NaN
1,00ddcc72af874a04a11e2735b1524b39,MP-1 height,"MP assumed to be the top of casing, not protec...",473358095061401,NaN,True,1984-10-17T00:00:00.0000000-06:00,ft,3.0,NaN,NaN,FromTopToBottom,From GW_MPNT 1,2021-01-12T20:13:22.9820002-06:00,admin,NaN,NaN


(577, 17)

In [11]:
# Drop the row if the "UniqueId" value is a duplicate. Drop the whole "StandardIdentifier" column
df_rp2 = df_rp.drop_duplicates(subset=['UniqueId']).drop("StandardIdentifier", axis=1)
display(df_rp2.head(2), df_rp2.shape)  #[573 rows x 16 columns]

,UniqueId,Name,Description,site,IsMeasuredAgainstLocalAssumedDatum,ValidFrom,Unit,Elevation,Uncertainty,Method,MeasurementDirection,Comment,AppliedTime,AppliedByUser,DecommissionedDate,DecommissionedReason
0,0868303bbc6c441982fff320c2d0a44f,MP-1 height,"No MP exists, but 0.00 ft. was populated for f...",473356095043701,True,0001-01-01T00:00:00.0000000+00:00,ft,0.0,0.05,GNSS3,FromTopToBottom,"No MP exists, but 0.00 ft. was populated for f...",2021-03-02T13:12:52.2229750-06:00,aberg,NaN,NaN
1,00ddcc72af874a04a11e2735b1524b39,MP-1 height,"MP assumed to be the top of casing, not protec...",473358095061401,True,1984-10-17T00:00:00.0000000-06:00,ft,3.0,NaN,NaN,FromTopToBottom,From GW_MPNT 1,2021-01-12T20:13:22.9820002-06:00,admin,NaN,NaN


(577, 16)

In [12]:
df_rp_newest =  df_rp2.sort_values('ValidFrom').drop_duplicates('site', keep='last')

In [13]:
df_rp2.to_csv("data_inputs/aquarius/Referencepoints_fromPy.csv", index=False)
df_rp_newest.to_csv('data_inputs/aquarius/Referencepoints_updatedMP_fromPy.csv', index=False)

# MLR data
Using water services websites

Alternatively you can use the USGS website to plug in specific site numbers: https://waterservices.usgs.gov/rest/Site-Test-Tool.html

In [14]:
# Getting site list straight from the database: 
## Gbe_db = pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=P:\0083\analysis\DataCompilation\DataCompilationPy\local_access_db\BemidjiMasterSiteData_be.accdb;')
## Gfe_db = pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=P:\0083\analysis\DataCompilation\DataCompilationPy\local_access_db\BemidjiMasterSiteData_fe.accdb;')
Gfe_db = pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=P:\0083\analysis\DataCompilation\BemidjiMasterSiteData_fe.accdb;')
c_fe = Gfe_db.cursor()
c_fe.execute('select USGS_siteno from tblSites')
df = pd.read_sql('select USGS_siteno from tblSites', Gfe_db)

#Close database connection and cursor
c_fe.close()
Gfe_db.close()

C:\Users\bmilinic\AppData\Local\Temp\1\ipykernel_36400\3561433712.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('select USGS_siteno from tblSites', Gfe_db)


In [51]:
# Drop none values, create a list out of the column, and create a string out of the column
df_sites = df.dropna()
sites = df_sites['USGS_siteno'].tolist()
sites = ', '.join(map(str,sites))

#Create URL that can be copy and pasted/clicked : can also be accessed through https://waterservices.usgs.gov/rest/Site-Test-Tool.html
url = f"https://waterservices.usgs.gov/nwis/site/?format=rdb&sites={sites}&siteOutput=expanded&siteStatus=all"
url = url.replace(" ", "%20")
print(url[0:100], 'and so on')

https://waterservices.usgs.gov/nwis/site/?format=rdb&sites=473429095051006,%20473424095052912,%20473 and so on


Beautiful Soup

In [18]:
# Read the webpage
r = urllib.request.urlopen(url).read()

# Parse it into components (title, tables, etc., with the Python module lxml) (turned out to be less useful since all the data is in one section)
#soup = BeautifulSoup(r, "lxml")
soup = BeautifulSoup(r, 'html.parser')

In [19]:
string = soup.string #made one giant string
lines = string.split("\n")# split the string into seperate lines using \n

In [20]:
# creating the dataframe
n=1
for x in lines:  # looping through every element in the list "line"
    words = x.split("\t") # split the line into works
    if words[0] == "agency_cd": # get column names from the line that starts with agency_cd
        col_names=words
    if words[0] == "USGS": 
        if n == 1:
            dfmlr = pd.DataFrame([words], columns=col_names) # use the first line to create a df
            n += 1
        else: 
            dfmlr.loc[len(dfmlr)] = words  # second line and so on gets added to the df

In [21]:
dfmlr

,agency_cd,site_no,station_nm,site_tp_cd,lat_va,long_va,dec_lat_va,dec_long_va,coord_meth_cd,coord_acy_cd,...,local_time_fg,reliability_cd,gw_file_cd,nat_aqfr_cd,aqfr_cd,aqfr_type_cd,well_depth_va,hole_depth_va,depth_src_cd,project_no
0,USGS,473356095043701,"GRANT CREEK, AT ATV TRAIL, SE OF PINEWOOD, MN",ST,473355.55,0950437.46,47.56543056,-95.0770722,X,C,...,Y,C,YY Y,,,,,,,MN-09500
1,USGS,473358095061401,426 147N35W10ADCB 01,GW,473358.11,0950613.60,47.56614167,-95.1037778,S,1,...,Y,C,Y Y,N100GLCIAL,112GSSG,U,3.23,,S,MN-09500
2,USGS,473404095054101,409 147N35W11BCAB 01,GW,473404.31,0950541.39,47.56786389,-95.0948306,S,1,...,Y,C,Y Y,N100GLCIAL,112GSSG,U,12.13,,S,MN-09500
3,USGS,473405095060101,402 147N35W10ADAB 01,GW,473404.56,0950601.45,47.5679333,-95.1004028,S,1,...,Y,C,Y Y,N100GLCIAL,112GSSG,U,12.37,,S,MN-09500
4,USGS,473405095060201,525 147N35W10ADAB 01 000027...,GW,473406.66,0950558.93,47.56851667,-95.0997028,X,C,...,Y,C,YY Y,N100GLCIAL,112GSSG,U,43.16,,S,MN-09500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
608,USGS,473437095050101,"12LKNK2 147N35W02 01 on N Sh, Unnamed Lk",GW-TH,473437.3,0950501.3,47.57702778,-95.0836944,G,S,...,Y,C,Y,,,,1.90,1.90,S,MN-09500
609,USGS,473437095052401,524 147N35W02CADB 01 000027...,GW,473436.19,0950523.83,47.5767194,-95.0899528,X,C,...,Y,C,YY Y,N100GLCIAL,112GSSG,U,72.12,,S,MN-09500
610,USGS,473439095063001,"UNNAMED LAKE, E SIDE, AT TRAIL, SE OF PINEWOOD...",LK,473439.55,0950630.72,47.57765278,-95.1085333,X,H,...,Y,C,,,,,,,,MN-09500
611,USGS,473440095063001,427 147N35W03DBBD 01,GW,473439.7,0950629.80,47.5776944,-95.1082778,S,1,...,Y,C,Y Y,N100GLCIAL,112GSSG,U,4.03,,S,MN-09500


In [22]:
dfmlr.to_csv("data_inputs/MLR/MLR_fromPy.csv", index=False)

# Water Level Retrieval using the Water Services website
- now, data is just retrieved straight from NWIS so this part is not needed

In [23]:
# sites have already been retrieved in the code before and stored in the variable called "sites"
print(sites[0:100])

473429095051006, 473424095052912, 473424095052906, 473423095052902, 473412095050801, 473416095051301


TAB DELINEATED

## URL

In [24]:
# water level meta data 
## Recreate the URL 

# Using the same "sites" variable created before and new URL link...
url = f"https://waterservices.usgs.gov/nwis/gwlevels/?format=rdb&sites={sites}&siteStatus=all"
url = url.replace(" ", "%20")
print(url[0:100])

https://waterservices.usgs.gov/nwis/gwlevels/?format=rdb&sites=473429095051006,%20473424095052912,%2


In [ ]:
# URL NEEDS UPDATED: https://waterservices.usgs.gov/test-tools/?service=gwlevels&siteType=&statTypeCd=all&major-filters=sites&format=json&date-type=type-none&statReportType=daily&statYearType=calendar&missingData=off&siteStatus=all&siteNameMatchOperator=start


In [29]:
print(url)

https://waterservices.usgs.gov/nwis/gwlevels/?format=rdb&sites=473429095051006,%20473424095052912,%20473424095052906,%20473423095052902,%20473412095050801,%20473416095051301,%20473431095052801,%20473356095043701,%20473419095052504,%20473408095045601,%20473426095051001,%20473426095051002,%20473426095051003,%20473425095052706,%20473425095052707,%20473425095052708,%20473425095052709,%20473425095052710,%20473425095052711,%20473425095052712,%20473425095052713,%20473425095052714,%20473426095052422,%20473426095052423,%20473426095052424,%20473426095052425,%20473426095052426,%20473426095052427,%20473426095052428,%20473425095052603,%20473426095052604,%20473426095052605,%20473426095052606,%20473425095052606,%20473425095052607,%20473425095052608,%20473425095052609,%20473425095052502,%20473425095052503,%20473426095052429,%20473426095052430,%20473426095052446,%20473426095052447,%20473426095052615,%20473426095052616,%20473426095052617,%20473426095052618,%20473426095052619,%20473426095052620,%20473426

In [ ]:
# Read the webpage
r = urllib.request.urlopen(url).read()

# Parse it into components (title, tables, etc., with the Python module lxml) (turned out to be less useful since all the data is in one section)
soup = BeautifulSoup(r, "lxml")

string = soup.string #made one giant string
lines = string.split("\n")# split the string into seperate lines using \n

HTTPError: HTTP Error 503: Service Unavailable

In [9]:
# creating the water level dataframe
n=1
for x in lines:  # looping through every element in the list "line"
    words = x.split("\t") # split the line into words
    if words[0] == "agency_cd": # get column names from the line that starts with agency_cd
        col_names=words
    if words[0] == "USGS": 
        if n == 1:
            dfwl = pd.DataFrame([words], columns=col_names) # use the first line to create a df
            n += 1
        else: 
            dfwl.loc[len(dfwl)] = words  # second line and so on gets added to the df

In [10]:
display(dfwl.head(2), dfwl.shape)

,agency_cd,site_no,site_tp_cd,lev_dt,lev_tm,lev_tz_cd,lev_va,sl_lev_va,sl_datum_cd,lev_status_cd,lev_agency_cd,lev_dt_acy_cd,lev_acy_cd,lev_src_cd,lev_meth_cd,lev_age_cd
0,USGS,473356095043701,ST,2022-08-25,14:08,UTC,-1.06,,,,USGS,m,,S,D,P
1,USGS,473358095061401,GW,1992-06-16,,UTC,0.00,,,1,USGS,D,,S,V,A
2,USGS,473404095054101,GW,2000-10-17,,UTC,10.33,,,1,USGS,D,,S,V,A
3,USGS,473405095060101,GW,2016-08-11,14:30,UTC,9.42,,,1,USGS,m,,S,V,A
4,USGS,473405095060201,GW,2016-08-11,14:29,UTC,9.39,,,1,USGS,m,,S,V,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305,USGS,473435095061501,GW,2022-10-28,15:09,UTC,23.33,,,1,USGS,m,,S,V,P
306,USGS,473437095052401,GW,2022-10-28,15:28,UTC,35.00,,,1,USGS,m,,S,V,P
307,USGS,473439095063001,LK,2022-08-25,16:17,UTC,-0.92,,,,USGS,m,,S,D,P
308,USGS,473440095063001,GW,1992-06-16,,UTC,1.60,,,1,USGS,D,,S,V,A


## dataretrieval

In [52]:
# convert the sites string into a list
sites = sites.split(', ')

In [60]:
display(sites[:3])
display(len(sites))

['473429095051006', '473424095052912', '473424095052906']

614

In [70]:
# Retrieve groundwater level data
dfwl = nwis.get_record(sites=sites[:400], service="gwlevels", siteStatus="all")

# Display the first few rows
dfwl.head()

c:\Users\bmilinic\AppData\Local\miniforge3\envs\bem_env2\Lib\site-packages\dataretrieval\nwis.py:1489: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(
c:\Users\bmilinic\AppData\Local\miniforge3\envs\bem_env2\Lib\site-packages\dataretrieval\utils.py:90: UserWarning: Warning: 27990 incomplete dates found, consider setting datetime_index to False.
  warnings.warn(


agency_cd site_tp_cd      lev_dt  \
site_no         datetime                                                     
473356095043701 2019-06-28 14:56:00+00:00      USGS         ST  2019-06-28   
                2019-06-28 14:56:00+00:00      USGS         ST  2019-06-28   
                2020-08-06 18:00:00+00:00      USGS         ST  2020-08-06   
                2020-08-06 18:00:00+00:00      USGS         ST  2020-08-06   
                2021-08-19 16:00:00+00:00      USGS         ST  2021-08-19   

                                          lev_tm lev_tz_cd  lev_va  sl_lev_va  \
site_no         datetime                                                        
473356095043701 2019-06-28 14:56:00+00:00  14:56     +0000     NaN    1390.17   
                2019-06-28 14:56:00+00:00  14:56     +0000     0.0        NaN   
                2020-08-06 18:00:00+00:00  18:00     +0000     NaN    1391.77   
                2020-08-06 18:00:00+00:00  18:00     +0000    -1.6        NaN   
                2021-08-19 16:00:00+00:00  16:00     +0000     NaN    1390.33   

                                          sl_datum_cd lev_status_cd  \
site_no         datetime                                              
473356095043701 2019-06-28 14:56:00+00:00      NAVD88           NaN   
                2019-06-28 14:56:00+00:00         NaN           NaN   
                2020-08-06 18:00:00+00:00      NAVD88           NaN   
                2020-08-06 18:00:00+00:00         NaN           NaN   
                2021-08-19 16:00:00+00:00      NAVD88           NaN   

                                          lev_agency_cd lev_dt_acy_cd  \
site_no         datetime                                                
473356095043701 2019-06-28 14:56:00+00:00          USGS             m   
                2019-06-28 14:56:00+00:00          USGS             m   
                2020-08-06 18:00:00+00:00          USGS             m   
                2020-08-06 18:00:00+00:00          USGS             m   
                2021-08-19 16:00:00+00:00          USGS             m   

                                          lev_acy_cd lev_src_cd lev_meth_cd  \
site_no         datetime                                                      
473356095043701 2019-06-28 14:56:00+00:00          1          S           D   
                2019-06-28 14:56:00+00:00          1          S           D   
                2020-08-06 18:00:00+00:00          1          S           D   
                2020-08-06 18:00:00+00:00          1          S           D   
                2021-08-19 16:00:00+00:00          0          S           D   

                                          lev_age_cd parameter_cd  
site_no         datetime                                           
473356095043701 2019-06-28 14:56:00+00:00          A        62611  
                2019-06-28 14:56:00+00:00          A        72019  
                2020-08-06 18:00:00+00:00          A        62611  
                2020-08-06 18:00:00+00:00          A        72019  
                2021-08-19 16:00:00+00:00          A        62611

The retrieval is too big...so it needs to be chopped up

In [87]:
# lets do chunks of 400
chunks = 400
# now many chunks are needed?
n = int(np.ceil(len(sites) / chunks)) # int is needed for np.array_split
print(len(sites), '/', chunks, 'rounds up to', n)
# split the sites into groups
split_sites_list = np.array_split(sites, n)

# PULL DATA IN GROUPS
data_list = []
for sites_chunk in split_sites_list:
    wl_data = nwis.get_record(sites=sites_chunk.tolist(), 
                              service="gwlevels", 
                              siteStatus="all")
    data_list.append(wl_data)

614 / 400 rounds up to 2


c:\Users\bmilinic\AppData\Local\miniforge3\envs\bem_env2\Lib\site-packages\dataretrieval\utils.py:90: UserWarning: Warning: 12264 incomplete dates found, consider setting datetime_index to False.
  warnings.warn(
c:\Users\bmilinic\AppData\Local\miniforge3\envs\bem_env2\Lib\site-packages\dataretrieval\utils.py:90: UserWarning: Warning: 38751 incomplete dates found, consider setting datetime_index to False.
  warnings.warn(


In [94]:
dfwl = pd.concat(data_list)

In [95]:
type(data_list[0])

pandas.core.frame.DataFrame

In [96]:
dfwl.iloc[:, 12].unique()

array(['S', nan], dtype=object)

## export

In [97]:
dfwl.to_csv("../create_master_oil_levels/data_inputs/NWISwaterLevel_fromPy.csv")#, index=False)